In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rpreactor

rpreactor.__version__

'0.0.12+61.g12dad48.dirty'

# Basic library usage

## Rules and chemicals: load and store

`rpreactor` uses a file database (SQLite3) to store:

* reaction rules
* and molecules

This has the advantage to be faster that plain text, more memory-efficient than always loading everything in-memory, and more lightweight than using a all-included relational database.
You should be able to use rpreactor on rather large datasets.

For the examples below, we will create simple databases with one rule and one metabolite defined as:

In [3]:
inchi = "InChI=1S/C12H22O11/c13-1-3-5(15)6(16)9(19)12(22-3)23-10-4(2-14)21-11(20)8(18)7(10)17/h3-20H,1-2H2/t3-,4-,5+,6+,7-,8-,9-,10-,11?,12+/m1/s1"
rsmarts = "([#6@@&v4:1]1(-[#8&v2:2]-[#1&v1:3])(-[#1&v1:4])-[#6@&v4:5](-[#8&v2:6]-[#1&v1:7])(-[#1&v1:8])-[#6@&v4:9](-[#8&v2:10]-[#1&v1:11])(-[#1&v1:12])-[#6@&v4:13](-[#6&v4:14](-[#8&v2:15]-[#1&v1:16])(-[#1&v1:17])-[#1&v1:18])(-[#1&v1:19])-[#8&v2:20]-[#6@&v4:21]-1(-[#8&v2:22]-[#6@@&v4:23]1(-[#1&v1:24])-[#6@@&v4:25](-[#8&v2:26]-[#1&v1:27])(-[#1&v1:28])-[#6@&v4:29](-[#8&v2:30]-[#1&v1:31])(-[#1&v1:32])-[#6&v4:33](-[#8&v2:34]-[#1&v1:35])(-[#1&v1:36])-[#8&v2:37]-[#6@@&v4:38]-1(-[#6&v4:39](-[#8&v2:40]-[#1&v1:41])(-[#1&v1:42])-[#1&v1:43])-[#1&v1:44])-[#1&v1:45])>>([#6@@&v4:1]1(-[#8&v2:2]-[#1&v1:3])(-[#1&v1:4])-[#6@&v4:5](-[#8&v2:6]-[#1&v1:7])(-[#1&v1:8])-[#6@&v4:9](-[#8&v2:10]-[#1&v1:11])(-[#1&v1:12])-[#6@&v4:13](-[#6&v4:14](-[#8&v2:15]-[#1&v1:16])(-[#1&v1:17])-[#1&v1:18])(-[#1&v1:19])-[#8&v2:20]-[#6@&v4:21]-1(-[#8&v2:22]-[#1&v1])-[#1&v1:45].[#6@@&v4:29]1(-[#8&v2:30]-[#1&v1:31])(-[#1&v1:32])-[#6@&v4:25](-[#8&v2:26]-[#1&v1:27])(-[#1&v1:28])-[#6@@&v4:23](-[#8&v2]-[#1&v1])(-[#1&v1:24])-[#6@&v4:38](-[#6&v4:39](-[#8&v2:40]-[#1&v1:41])(-[#1&v1:42])-[#1&v1:43])(-[#1&v1:44])-[#8&v2:37]-[#6&v4:33]-1(-[#8&v2:34]-[#1&v1:35])-[#1&v1:36])"

### Using a throw-away in-memory database

In [4]:
# Create the database
o = rpreactor.RuleBurner(with_hs=True)

# Populate it
o.insert_inchi([inchi])
o.insert_rsmarts([rsmarts])

# Try all rules vs. all metabolites with rule_mol='*'
results = [x for x in o.compute(rule_mol='*')]
results

[{'rule_id': '0',
  'substrate_id': '0',
  'product_list': [[<rdkit.Chem.rdchem.Mol at 0x7ff378e8b1d0>,
  'product_inchikeys': [['WQZGKKKJIJFFOK-UHFFFAOYSA-N',
    'WQZGKKKJIJFFOK-UHFFFAOYSA-N']],
  'product_inchis': [['InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2/h2-11H,1H2',
    'InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2/h2-11H,1H2']],
  'product_smiles': [['[H]OC([H])([H])C1([H])OC([H])(O[H])C([H])(O[H])C([H])(O[H])C1([H])O[H]',
    '[H]OC([H])([H])C1([H])OC([H])(O[H])C([H])(O[H])C([H])(O[H])C1([H])O[H]']]}]

That's all.

Note that you may export the database as SQL with:

In [5]:
help(o.dump_to_sql)

Help on method dump_to_sql in module rpreactor.rule.burner:

dump_to_sql(path) method of rpreactor.rule.burner.RuleBurner instance
    Dump the database as a SQL file.
    
    :param path: Path to the SQL file.
    :type path: str



### About chemicals and rule identifiers

Notice that in previous example we provided molecules and rules as simple list and that rpreactor automatically attributed a 0-based identifier to each molecule and rule registered in the database. As we will see later on, those identifiers will be used to pick specific rule/molecules.

It is also possible to provide rules and molecules as Dict rather that List to choose a more user-friendly identifier:

In [6]:
o = rpreactor.RuleBurner(with_hs=True)

o.insert_inchi({'inchi1': inchi})
o.insert_rsmarts({'rsmart1': rsmarts})

results = [x for x in o.compute(rule_mol='*')]
results

[{'rule_id': 'rsmart1',
  'substrate_id': 'inchi1',
  'product_list': [[<rdkit.Chem.rdchem.Mol at 0x7ff378ef7a90>,
  'product_inchikeys': [['WQZGKKKJIJFFOK-UHFFFAOYSA-N',
    'WQZGKKKJIJFFOK-UHFFFAOYSA-N']],
  'product_inchis': [['InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2/h2-11H,1H2',
    'InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2/h2-11H,1H2']],
  'product_smiles': [['[H]OC([H])([H])C1([H])OC([H])(O[H])C([H])(O[H])C([H])(O[H])C1([H])O[H]',
    '[H]OC([H])([H])C1([H])OC([H])(O[H])C([H])(O[H])C([H])(O[H])C1([H])O[H]']]}]

The list of chemicals and rules identifiers in use are always available using the eponym attribute of RuleBurner:

In [7]:
o.rules

['rsmart1']

In [8]:
o.chemicals

['inchi1']

As we will see later on, it is useful to subselect some identifiers.

### Using a persitent file database (SQLite3)

Create a file database in current directory:

In [9]:
o = rpreactor.RuleBurner(db_path="TMP_showcase.sqlite3", with_hs=True)

o.insert_inchi([inchi])
o.insert_rsmarts([rsmarts])

results = [x for x in o.compute(rule_mol='*')]
results

[{'rule_id': '0',
  'substrate_id': '0',
  'product_list': [[<rdkit.Chem.rdchem.Mol at 0x7ff378eefd10>,
  'product_inchikeys': [['WQZGKKKJIJFFOK-UHFFFAOYSA-N',
    'WQZGKKKJIJFFOK-UHFFFAOYSA-N']],
  'product_inchis': [['InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2/h2-11H,1H2',
    'InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2/h2-11H,1H2']],
  'product_smiles': [['[H]OC([H])([H])C1([H])OC([H])(O[H])C([H])(O[H])C([H])(O[H])C1([H])O[H]',
    '[H]OC([H])([H])C1([H])OC([H])(O[H])C([H])(O[H])C([H])(O[H])C1([H])O[H]']]}]

File database can be exchanged and loaded at a later date. Let's check that the database is persistent:

In [10]:
del o
o = rpreactor.RuleBurner(db_path="TMP_showcase.sqlite3", with_hs=True)
results = [x for x in o.compute(rule_mol='*')]
results

[{'rule_id': '0',
  'substrate_id': '0',
  'product_list': [[<rdkit.Chem.rdchem.Mol at 0x7ff378e8ba40>,
  'product_inchikeys': [['WQZGKKKJIJFFOK-UHFFFAOYSA-N',
    'WQZGKKKJIJFFOK-UHFFFAOYSA-N']],
  'product_inchis': [['InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2/h2-11H,1H2',
    'InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2/h2-11H,1H2']],
  'product_smiles': [['[H]OC([H])([H])C1([H])OC([H])(O[H])C([H])(O[H])C([H])(O[H])C1([H])O[H]',
    '[H]OC([H])([H])C1([H])OC([H])(O[H])C([H])(O[H])C([H])(O[H])C1([H])O[H]']]}]

Note that the RDKit objects are different (i.e. not allocated to the same memory address). This is because we instanciated a new object using the serialized data in the database.

(Clean up our workspace:)

In [11]:
! rm TMP_showcase.sqlite3

### Create a database with all rules from RetroRules

A pregenerated set of reaction rules can be download from [RetroRules](https://retrorules.org). It contains information of all known enzymatic reactions from MetaNetX v3.0. `rpreactor` can load those reaction rules.

We will use a dedicated method to translate a RetroRules dataset into a rpreactor database:

In [12]:
help(rpreactor.create_db_from_retrorules)

Help on function create_db_from_retrorules in module rpreactor.rule.utils:

create_db_from_retrorules(path_retrosmarts_tsv, db_path, with_hs=False, with_stereo=False, version='v1.0')
    Convert a RetroRules dataset to a rpreactor-ready sqlite3 database.
    
    All rules and all molecules will be extracted from the TSV file and imported into the database.
    For more information on RetroRules, see https://retrorules.org/.



Of course, you will need to first get yourself a version of RetroRules.

In [13]:
path = "retrorules_rr02_rp3_hs/retrorules_rr02_flat_all.tsv"

In [14]:
%%time
! rm TMP_retrorules.sqlite3
rpreactor.create_db_from_retrorules(path_retrosmarts_tsv=path, db_path="TMP_retrorules.sqlite3", with_hs=True)

rm: cannot remove 'TMP_retrorules.sqlite3': No such file or directory
CPU times: user 7min 8s, sys: 38.7 s, total: 7min 46s
Wall time: 16min 56s


One of the advantages of using RetroRules is that it comes with preconfigured rules and results:

In [15]:
o = rpreactor.RuleBurner(db_path="TMP_retrorules.sqlite3", with_hs=True)

Here is the number of unique rules at each diameter:

In [16]:
import sqlite3

for diameter in [2, 4, 6, 8, 10, 12, 16]:
    ans = o.db.execute("select count(*) from rules where diameter=?;", [diameter]).fetchone()
    print(f"Found {ans[0]} rules at diameter {diameter}")

Found 19043 rules at diameter 2
Found 24806 rules at diameter 4
Found 28408 rules at diameter 6
Found 30104 rules at diameter 8
Found 31045 rules at diameter 10
Found 31699 rules at diameter 12
Found 32574 rules at diameter 16


## Apply reaction rules to metabolites

We will re-use the reaction rules and the metabolites from the RetroRules database:

In [17]:
o = rpreactor.RuleBurner(db_path="TMP_retrorules.sqlite3", with_hs=True)

In [18]:
help(rpreactor.RuleBurner.compute)

Help on function compute in module rpreactor.rule.burner:

compute(self, rule_mol, commit=False, max_workers=1, timeout=60, chunk_size=1000)
    Returns a generator over products predicted by applying rules on molecules.
    
    Important: rules and molecules must be inserted into the database beforehand and are referred to by their
    identifier. We assume each identifier refer to distinct rules and molecules: several identifiers referring to
    identical rules/molecules will be computed independently.
    
    :param rule_mol: Tasks to compute as in: [(<rule_id>, <mol_id>), ...]. If the magic argument "*" is provided,
        will compute all rules against all metabolites (mainly useful for debug purpose).
    :type rule_mol: list of tuple, or '*'
    :param commit: If true, the results be commited to the `results` table for later retrieval. Default: False.
    :type commit: bool
    :param max_workers: Maximum number of cores to use simultaneously to compute the tasks. Default: 1

### Some rules vs. some metabolites

So far, we used `rule_mol='*'` to specify that we wanted to compute all rules against all metabolites. This can be rather expensive on large dataset; thus it is better to specify which rules we want to apply on which molecule:

In [19]:
rule_list = ['RR-02-0622b5154c1abb9a-02-F', 'RR-02-84cfa33041e82595-02-F', 'RR-02-b93f1788ade7b1a8-02-F']
mol_list = ['MNXM102752', 'MNXM8160']

tasks = [(rule, mol) for rule in rule_list for mol in mol_list]

results = [x for x in o.compute(rule_mol=tasks)]
results

[{'rule_id': 'RR-02-0622b5154c1abb9a-02-F',
  'substrate_id': 'MNXM102752',
  'product_list': [[<rdkit.Chem.rdchem.Mol at 0x7ff35c7ee7c0>]],
  'product_inchikeys': [['SZCBXWMUOPQSOX-UHFFFAOYSA-N']],
  'product_inchis': [['InChI=1S/C40H56O4/c1-29(17-13-19-31(3)21-23-39-35(5,6)25-33(41)27-37(39,9)43-39)15-11-12-16-30(2)18-14-20-32(4)22-24-40-36(7,8)26-34(42)28-38(40,10)44-40/h11-24,33-34,41-42H,25-28H2,1-10H3']],
  'product_smiles': [['[H]OC1([H])C([H])([H])C2(C([H])([H])[H])OC2(C([H])=C([H])C(=C([H])C([H])=C([H])C(=C([H])C([H])=C([H])C([H])=C(C([H])=C([H])C([H])=C(C([H])=C([H])C23OC2(C([H])([H])[H])C([H])([H])C([H])(O[H])C([H])([H])C3(C([H])([H])[H])C([H])([H])[H])C([H])([H])[H])C([H])([H])[H])C([H])([H])[H])C([H])([H])[H])C(C([H])([H])[H])(C([H])([H])[H])C1([H])[H]']]},
 {'rule_id': 'RR-02-84cfa33041e82595-02-F',
  'substrate_id': 'MNXM8160',
  'product_list': [[<rdkit.Chem.rdchem.Mol at 0x7ff39618bea0>]],
  'product_inchikeys': [['MYMOFIZGZYHOMD-UHFFFAOYSA-N']],
  'product_inchis': [[

Following the same logic, you may want to apply lots of rules to some metabolites. The key here is to create a list of the reactions (or compounds) identifiers we want to use. You can use a direct access to the database to choose which rules you want to use or use some handy shortcuts exposed as properties of the RuleBurner object.

### Queries against all rules or all chemicals

We provide rulesand chemicals properties to access the list of all currently used identifiers. It is handy to query all rules (or all chemicals) against one chemical (or rule).

Be careful though, as this will be time consuming!

In [20]:
len(o.rules), len(o.chemicals)

(229862, 12490)

In [21]:
tasks = [('RR-02-0622b5154c1abb9a-02-F', mol) for mol in o.chemicals]
len(tasks)

12490

Moreover, as we have seen before, you can directly use rule_mol='*' to compute all rules against all chemicals.

### Informations about last execution

We prefer to catch the errors that may occurs in a compute run and log them rather to propagate the Exceptions. This way, you should not loose your work if the last task of a long run goes wrong. You can always ask for a summary of last executed run:

In [22]:
o.summary()

{'database_rules_count': 229862,
 'database_chemical_count': 12490,
 'database_results_count': 666144,
 'lastcompute_precomputed_count': 2,
 'lastcompute_newlycomputed_count': 1,
 'lastcompute_timeout_list': [],
 'lastcompute_errors_list': []}

This information is also available when you represent the RuleBurner object as a string:

In [23]:
print(o)

Connected to a database with 229862 rules, 12490 compounds, and 666144 results (at 'TMP_retrorules.sqlite3'). Last compute call yield 3 results (2 precomputed, 1 new); 0 errors were caught (details in the logs), and 0 timeouts were hit.


Of course, you should always have a quick look at the logs if you truely want to investigate what happened.

### Reuse results for subsequent calls

You may commit the results (i.e. the chemicals found after applying a rule to a chemical) to remember them the next time you will make the same query. This will greatly improve the time it takes to retrieve the results.

In [25]:
o = rpreactor.RuleBurner(with_hs=True)
o.insert_inchi([inchi])
o.insert_rsmarts([rsmarts])

tasks = [(rid, cid) for rid in o.rules for cid in o.chemicals]

Without using commit:

In [26]:
%%timeit
results = [x for x in o.compute(tasks, commit=False)]  # by default, commit=False

112 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Using commit, the first call should have pretty much the same cost, but subsequent calls should be much faster:

In [27]:
%%time
results = [x for x in o.compute(tasks, commit=True)]

CPU times: user 2.65 ms, sys: 23.2 ms, total: 25.8 ms
Wall time: 120 ms


In [28]:
%%time
results = [x for x in o.compute(tasks, commit=True)]

CPU times: user 982 µs, sys: 15 µs, total: 997 µs
Wall time: 700 µs


Notice that this will also insert chemicals in the database:

In [29]:
o.chemicals

['0', '1']

You may also use a method to clear the results if you don't need them or if you want to recompute them:

In [30]:
help(o.drop_results)

Help on method drop_results in module rpreactor.rule.burner:

drop_results() method of rpreactor.rule.burner.RuleBurner instance
    Drop the results table and computed metabolites.



## Conversions of output format

In [31]:
o = rpreactor.RuleBurner(db_path="TMP_retrorules.sqlite3", with_hs=True)

In [32]:
rule_list = ['RR-02-0622b5154c1abb9a-02-F', 'RR-02-84cfa33041e82595-02-F', 'RR-02-b93f1788ade7b1a8-02-F']
mol_list = ['MNXM102752', 'MNXM8160', 'MNXM2798']

tasks = [(rule, mol) for rule in rule_list for mol in mol_list]

results = [x for x in o.compute(tasks)]

### to JSON

In [33]:
import json

# Trim-out the RDKit objects under the "product_list" key
results_json = [{k:v for k,v in d.items() if k != 'product_list'} for d in results]

json.dumps(results_json)

'[{"rule_id": "RR-02-0622b5154c1abb9a-02-F", "substrate_id": "MNXM102752", "product_inchikeys": [["SZCBXWMUOPQSOX-UHFFFAOYSA-N"]], "product_inchis": [["InChI=1S/C40H56O4/c1-29(17-13-19-31(3)21-23-39-35(5,6)25-33(41)27-37(39,9)43-39)15-11-12-16-30(2)18-14-20-32(4)22-24-40-36(7,8)26-34(42)28-38(40,10)44-40/h11-24,33-34,41-42H,25-28H2,1-10H3"]], "product_smiles": [["[H]OC1([H])C([H])([H])C2(C([H])([H])[H])OC2(C([H])=C([H])C(=C([H])C([H])=C([H])C(=C([H])C([H])=C([H])C([H])=C(C([H])=C([H])C([H])=C(C([H])=C([H])C23OC2(C([H])([H])[H])C([H])([H])C([H])(O[H])C([H])([H])C3(C([H])([H])[H])C([H])([H])[H])C([H])([H])[H])C([H])([H])[H])C([H])([H])[H])C([H])([H])[H])C(C([H])([H])[H])(C([H])([H])[H])C1([H])[H]"]]}, {"rule_id": "RR-02-84cfa33041e82595-02-F", "substrate_id": "MNXM8160", "product_inchikeys": [["MYMOFIZGZYHOMD-UHFFFAOYSA-N"]], "product_inchis": [["InChI=1S/O2/c1-2"]], "product_smiles": [["O=O"]]}, {"rule_id": "RR-02-b93f1788ade7b1a8-02-F", "substrate_id": "MNXM2798", "product_inchikeys": 

### to pandas DataFrame

In [34]:
import pandas as pd

df = pd.DataFrame.from_dict(results)
df

,rule_id,substrate_id,product_list,product_inchikeys,product_inchis,product_smiles
0,RR-02-0622b5154c1abb9a-02-F,MNXM102752,[[<rdkit.Chem.rdchem.Mol object at 0x7ff35b71a...,[[SZCBXWMUOPQSOX-UHFFFAOYSA-N]],[[InChI=1S/C40H56O4/c1-29(17-13-19-31(3)21-23-...,[[[H]OC1([H])C([H])([H])C2(C([H])([H])[H])OC2(...
1,RR-02-84cfa33041e82595-02-F,MNXM8160,[[<rdkit.Chem.rdchem.Mol object at 0x7ff362ca8...,[[MYMOFIZGZYHOMD-UHFFFAOYSA-N]],[[InChI=1S/O2/c1-2]],[[O=O]]
2,RR-02-b93f1788ade7b1a8-02-F,MNXM2798,[[<rdkit.Chem.rdchem.Mol object at 0x7ff362ca8...,[[VYKLRWGPNUVKNC-UHFFFAOYSA-N]],"[[InChI=1S/C13H20O3/c1-9(14)5-6-13-11(2,3)7-10...",[[[H]OC1([H])C([H])([H])C2(C([H])([H])[H])OC2(...
3,RR-02-b93f1788ade7b1a8-02-F,MNXM8160,[[<rdkit.Chem.rdchem.Mol object at 0x7ff355e8d...,"[[QGXAEWUQPKUEQA-UHFFFAOYSA-N, HLZZDTVXNYNFKH-...",[[InChI=1S/C18H26O3/c1-13(7-6-8-14(2)19)9-10-1...,[[[H]OC1([H])C([H])([H])C2(C([H])([H])[H])OC2(...


In [35]:
tmp = df.loc[(df.rule_id=='RR-02-0622b5154c1abb9a-02-F') & (df.substrate_id=='MNXM102752'), 'product_inchikeys']
tmp

0    [[SZCBXWMUOPQSOX-UHFFFAOYSA-N]]
Name: product_inchikeys, dtype: object

In [36]:
tmp[0]

[['SZCBXWMUOPQSOX-UHFFFAOYSA-N']]

In [37]:
tmp[0][0]

['SZCBXWMUOPQSOX-UHFFFAOYSA-N']

In [38]:
tmp[0][0][0]

'SZCBXWMUOPQSOX-UHFFFAOYSA-N'